In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

from data_generator import DataGenerator

import model as model_util

from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from numpy import array
import numpy as np


import datetime
from datetime import datetime

from tensorflow.keras.layers import BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences


from tensorflow.keras import datasets, layers, models

import tensorflow as tf

#%load_ext tensorboard
print(tf.__version__)

2.4.1


In [2]:
def reset_seed():
    seed_value= 0
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    import numpy as np
    np.random.seed(seed_value)
    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    tf.random.set_seed(seed_value)
    

In [8]:
import random

path = "drive/My Drive/model/"
path = "./"

trainX = np.load(path + "train" + "X.npy")
trainY = np.load(path + "train" + "Y.npy")
print("Train")
print(trainX.shape)
print(trainY.shape)

valX = np.load(path + "val" + "X.npy")
valY = np.load(path + "val" + "Y.npy")
print("Validation")
print(valX.shape)
print(valY.shape)

def get_balanced_set(X, Y):
    positiveX = []
    positiveY = []
    negativeX = []
    negativeY = []

    for i in tqdm(range(1, len(X)-1)):
        x = X[i]
        y = Y[i]
        if (y >= 0):
            positiveX.append(x)
            positiveY.append(y)
        else:
            negativeX.append(x)
            negativeY.append(y)
   
    positiveX = np.array(positiveX)
    positiveY = np.array(positiveY)
    negativeX = np.array(negativeX)
    negativeY = np.array(negativeY)
    
    print("Positives: {} Negatives {}".format(len(positiveX), len(negativeX)))
    
    negative_num = len(positiveY)
    start_index = random.randint(0,(len(negativeX) - negative_num))
    end_index = start_index + negative_num
    
    print(start_index)
    print(end_index)
    
    trainX = np.concatenate((positiveX, negativeX[start_index:end_index]), axis=0)
    trainY = np.concatenate((positiveY, negativeY[start_index:end_index]), axis=0)
    return trainX, trainY, positiveX, positiveY, negativeX, negativeY


#trainX, trainY, positiveX, positiveY, negativeX, negativeY = get_balanced_set(trainX, trainY)

#valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = get_balanced_set(valX, valY)


#print("positiveX {0}".format(positiveX.shape))
#print("negativeX {0}".format(negativeX.shape))

print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)

Train
(883497, 33)
(883497,)
Validation
(2658, 33)
(2658,)
(883497, 33)
(883497,)
(2658, 33)
(2658,)


In [10]:
# define generator
seqs = 20

class CustomGen(TimeseriesGenerator):
    def __getitem__(self, idx):
        x,y = super().__getitem__(idx)
        return x, y

def get_gen(set_x, set_y, shuffle=True, batch_size=512*2):
    return CustomGen(set_x, set_y, length=seqs, batch_size=batch_size, shuffle=shuffle)

train_generator = get_gen(trainX, trainY, shuffle=True)

val_generator = get_gen(valX, valY, shuffle=False)

# number of samples
print('Samples: %d' % len(train_generator))

x, y = train_generator[0]

x_, y_ = val_generator[0]


print(x.shape)
print(y.shape)

features = x.shape[-1]

print("Feature: {}".format(features))


Samples: 863
(1024, 20, 33)
(1024,)
Feature: 33


In [11]:
from numpy import array
import numpy as np

import datetime
from datetime import datetime

from tensorflow.keras.layers import RepeatVector, TimeDistributed, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input

from tensorflow.keras import Model, Sequential, regularizers

from tensorflow.keras import datasets, layers, models

import tensorflow as tf

dim = 250

def add_deep_layers(input_layer, units = dim):
    x = Dense(units, activation='relu')(input_layer)
    x = Dropout(0.2)(x)
    x = Dense(units, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(units, activation='relu')(x)
    return x

def get_model(seqs, features):
    reset_seed()
    inputX = Input(shape=(
        seqs,
        features
        )
    )
    x = inputX
    x = Dense(dim, activation='sigmoid')(x)
    x = LSTM(units=dim, return_sequences=True, activation='relu')(x)
    x = add_deep_layers(x, dim)
    x = LSTM(dim, activation='elu', return_sequences=False)(x)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, features)
    x = Dense(features, name= "encoder")(x)
    x = Dense(1, name='output')(x)

    model = Model(inputs=[inputX], outputs=x)
    
    return model

    
def load_model(filepath = "drive/My Drive/model/stock.h5"):
    model_loaded = tf.keras.models.load_model(filepath)
    return model_loaded

In [12]:


#model = load_model(filepath = "drive/My Drive/model/stock.h5")
#model.summary()

#!rm "drive/My Drive/model/encoder.h5"

model = get_model(seqs, features)

model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07),
    loss=tf.losses.MeanSquaredError(),
    metrics=['accuracy', tf.metrics.MeanAbsoluteError()]
)


In [13]:

filepath_encoder = "encoder.h5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath_encoder, monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = []

history = model.fit(
    #trainX, 
    #trainY,
    train_generator,
    validation_data=val_generator,
    #validation_data=[valX, valY],
    #validation_split=0.3,
    steps_per_epoch=len(train_generator)-1, 
    epochs=10, 
    verbose=1,
    callbacks=callbacks_list)


Epoch 1/10
862/862 [==============================] - 1165s 1s/step - loss: 2.9967e-06 - accuracy: 0.0043 - mean_absolute_error: 0.0013 - val_loss: 8.9173e-07 - val_accuracy: 0.0061 - val_mean_absolute_error: 8.2490e-04
Epoch 2/10
106/862 [==>...........................] - ETA: 1:36:13 - loss: 1.3184e-06 - accuracy: 0.0042 - mean_absolute_error: 8.9033e-04

KeyboardInterrupt: 

In [ ]:
def show_loss(history): 
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

show_loss(history)

In [ ]:
def show_accuracy(history):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

show_accuracy(history)